<a href="https://colab.research.google.com/github/eel-eel-eel/ric1340/blob/main/ch06_02_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 固有表現抽出の練習問題

## データ準備・前処理

In [ ]:
!wget https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/chABSA-dataset.zip
!unzip chABSA-dataset.zip
!mkdir -p /content/drive/MyDrive/bert/6_2_chABSA
!cp -r chABSA-dataset /content/drive/MyDrive/bert/6_2_chABSA

--2022-03-30 12:34:18--  https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/chABSA-dataset.zip
Resolving s3-ap-northeast-1.amazonaws.com (s3-ap-northeast-1.amazonaws.com)... 52.219.136.146
Connecting to s3-ap-northeast-1.amazonaws.com (s3-ap-northeast-1.amazonaws.com)|52.219.136.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 722777 (706K) [application/zip]
Saving to: ‘chABSA-dataset.zip’

chABSA-dataset.zip  100%[===================>] 705.84K  1007KB/s    in 0.7s    

2022-03-30 12:34:20 (1007 KB/s) - ‘chABSA-dataset.zip’ saved [722777/722777]

Archive:  chABSA-dataset.zip
   creating: chABSA-dataset/
  inflating: chABSA-dataset/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/chABSA-dataset/
  inflating: __MACOSX/chABSA-dataset/._.DS_Store  
 extracting: chABSA-dataset/.gitkeep  
  inflating: chABSA-dataset/e00008_ann.json  
  inflating: chABSA-dataset/e00017_ann.json  
  inflating: chABSA-dataset/e00024_ann.json  
  inflati

In [ ]:
label_kinds = []

# make labels (exclude NULL and OOD)
for e in ["market", "company", "business", "product"]:
    for a in ["general", "sales", "profit", "amount", "price", "cost"]:
        label_kinds.append(e + "#" + a)
        if e in ["market"]:
            break;

print(label_kinds)

['market#general', 'company#general', 'company#sales', 'company#profit', 'company#amount', 'company#price', 'company#cost', 'business#general', 'business#sales', 'business#profit', 'business#amount', 'business#price', 'business#cost', 'product#general', 'product#sales', 'product#profit', 'product#amount', 'product#price', 'product#cost']


In [ ]:
import json
import glob
import pandas as pd
from collections import Counter
import re
# from janome.tokenizer import Tokenizer


dataset = []
labels = []

for f in glob.glob("/content/drive/MyDrive/bert/6_2_chABSA/chABSA-dataset/e*_ann.json"):
    with open(f, encoding="utf-8") as j:
        d = json.load(j)
        for s in d["sentences"]:
            os = []
            cs = []
            ws = []
            from_to = []
            temp_dict = []
            for o in s["opinions"]:
                category = o["category"]
                word = o["target"]

                if category in label_kinds and category not in cs:
                  cs.append(category)
                  category = re.sub("#\D*", "", o["category"])
                  temp_dict2 = dict(name=word, span=[o["from"],o["to"]],type=category)
                  temp_dict.append(temp_dict2)

            if len(cs) > 0:
                dataset.append(
                    {"sentence": s["sentence"],
                     "entities": temp_dict}
                )
                # labels.append(os)

dataset = pd.DataFrame(dataset)
print(dataset.head(5))

                                            sentence  \
0  同事業の販売状況につきましては、自動車部品向けや、中国販売子会社での販売増により、販売数量は...   
1  電子部品向け金属粉の販売状況につきましては、スマートフォン関連需要に加え、軟磁性材向けの販売...   
2  粉末冶金向け金属粉の販売状況につきましては、自動車部品向けが好調な一方、熊本地震等の影響によ...   
3  セグメント利益は、電子部品向け金属粉の販売好調により、同186百万円増益（32.6％増益）の...   
4  不動産賃貸事業の当連結会計年度の売上高は23百万円（前年度比18.5%増収）、セグメント利益...   

                                            entities  
0  [{'name': '自動車部品向け', 'span': [16, 23], 'type':...  
1  [{'name': '金属粉', 'span': [6, 9], 'type': 'prod...  
2  [{'name': '金属粉', 'span': [6, 9], 'type': 'prod...  
3  [{'name': '金属粉', 'span': [15, 18], 'type': 'pr...  
4  [{'name': '不動産賃貸事業', 'span': [0, 7], 'type': '...  
[]


In [ ]:
dataset.head

<bound method NDFrame.head of                                                sentence  \
0     同事業の販売状況につきましては、自動車部品向けや、中国販売子会社での販売増により、販売数量は...   
1     電子部品向け金属粉の販売状況につきましては、スマートフォン関連需要に加え、軟磁性材向けの販売...   
2     粉末冶金向け金属粉の販売状況につきましては、自動車部品向けが好調な一方、熊本地震等の影響によ...   
3     セグメント利益は、電子部品向け金属粉の販売好調により、同186百万円増益（32.6％増益）の...   
4     不動産賃貸事業の当連結会計年度の売上高は23百万円（前年度比18.5%増収）、セグメント利益...   
...                                                 ...   
1339  当連結会計年度におきましては、連結子会社のデジタル・アドバタイジング・コンソーシアム株式会社...   
1340  新規の自動ドアの売上台数は僅かに減少したものの、シートシャッターの大型物件に加え、取替の売上...   
1341     加えて、保守契約が堅調に増加し、売上高は6,952百万円（前年同期比1.2％増）となりました   
1342  利益につきましては、取替工事の増加及び保守契約による安定的な利益の確保により、セグメント利益...   
1343  その他のセグメントでは駐輪システムが堅調に推移し、売上高は721百万円（前年同期比0.8％増...   

                                               entities  
0     [{'name': '自動車部品向け', 'span': [16, 23], 'type':...  
1     [{'name': '金属粉', 'span': [6, 9], 'type': 'prod...  
2     [{'name': '金属粉', 'span': [6, 9], 'type': 'prod...  
3     [{'name': '金属粉', 'span'

In [ ]:
max_len = 0

for i in range(dataset.shape[0]):
  if max_len < len(dataset.loc[i, "sentence"]):
    max_len = len(dataset.loc[i, "sentence"])
print("最大文字数：",max_len)

最大文字数： 334


In [ ]:
!pip install transformers[ja]

     |████████████████████████████████| 2.8 MB 5.2 MB/s 
     |████████████████████████████████| 895 kB 44.9 MB/s 
     |████████████████████████████████| 3.3 MB 27.5 MB/s 
     |████████████████████████████████| 596 kB 45.3 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 568 kB 45.0 MB/s 
     |████████████████████████████████| 13.4 MB 23.6 MB/s 
     |████████████████████████████████| 47.4 MB 52.6 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=7f2e492f5979963599a19062d6f7ae6f3e692a25e1e98feb7c7bfa63040adac7
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
  Created wheel for unidic: filename=unidic-1.1.0-py3-none-any.whl size=7426 sha256=b18759528e6f7c55fd60761b6b98b95cd548784ab458696991bd0a252c315f41
  Stored in directory: /root/.cache/pip/wheels/ce/4d/f1/170bb74b559ca338113c0315c9805e16dfd0a12411ec6b1122
  Created wheel for u

In [ ]:
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

In [ ]:
list_text = [dataset.loc[i,"sentence"] for i in range(dataset.shape[0])]
list_tokens = [tokenizer.tokenize(text) for text in list_text]
list_entities = [dataset.loc[i,"entities"] for i in range(dataset.shape[0])]

In [ ]:
# cl-tohoku/bert-base-japanese-whole-word-maskingのモデルは最大512トークンまで対応しているが、
# 学習時のGPUメモリ消費を抑えるため256としている
n_token = 256

# encode後のトークンは、特殊トークン（{CLS], {SEP]など）や特殊文字（##）が挿入されることに注意
# spanで表されているラベルと文字数ベースで位置ずれを起こすため、BIO形式に変換する際に補正する必要がある
list_text_id =  [tokenizer.encode(text, truncation=True, padding='max_length', max_length=n_token) for text in list_text]
list_tokens = [tokenizer.convert_ids_to_tokens(encode) for encode in list_text_id]

In [ ]:
def is_in_span(idx, span):
  return span[0] <= idx and idx < span[1]

In [ ]:
from collections import defaultdict
list_bio =[]

label2id = defaultdict(lambda :len(label2id))
_ = label2id["O"]

for text, tokens, entities in zip(list_text, list_tokens, list_entities):
  bio = ["O"] * len(tokens)

  for entity in entities:
    cnt = 0
    begin_flg = True
    label = entity["type"]

    for i, tok in enumerate(tokens):
      if tok == "[CLS]" or tok == "[SEP]" or tok == "[PAD]":
        continue
      elif is_in_span(cnt, entity["span"]):
        if begin_flg:
          bio[i] = f"B-{label}"
          begin_flg = False
          _ = label2id[f"B-{label}"]
          _ = label2id[f"I-{label}"]
        else:
          bio[i] = f"I-{label}"

      cnt += len(tok.replace("##", ""))

  list_bio.append(bio)

id2label = {v:k for k,v in label2id.items()}

In [ ]:
list_bio_id = [[label2id[label] for label in bio] for bio in list_bio]

In [ ]:
# BIO形式に変換できていることの確認
for dec, bio, _ in zip(list_tokens[0], list_bio[0], range(100)):
  print(f"{dec}\t{bio}")

[CLS]	O
同	O
事業	O
の	O
販売	O
状況	O
に	O
つき	O
まし	O
て	O
は	O
、	O
自動車	B-product
部品	I-product
向け	I-product
や	O
、	O
中国	B-company
販売	I-company
子会社	I-company
で	O
の	O
販売	O
増	O
により	O
、	O
販売	O
数量	O
は	O
前年度	O
比	O
69	O
##3	O
トン	O
増加	O
(	O
10	O
.	O
9	O
%	O
増加	O
)	O
の	O
7	O
,	O
05	O
##5	O
トン	O
と	O
なり	O
まし	O
た	O
[SEP]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O
[PAD]	O


In [ ]:
len(list_bio)

1344

In [ ]:
import torch

# 使用デバイスにGPUを設定
# 以下のような出力が出ていれば正常に設定ができている
# device(type='cuda', index=0)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
from torch.utils.data import Dataset

class NERDataset(Dataset):
  def __init__(self, texts_id, bios_id, is_test=False):
    self.texts_id = texts_id
    self.bios_id = bios_id
    self.is_test = is_test

  def __getitem__(self, idx):
    data = {'input_ids': torch.tensor(self.texts_id[idx], device=device)}
    if not self.is_test:
      data['label'] = torch.tensor(self.bios_id[idx], device=device)
    return data

  def __len__(self):
    return len(self.bios_id)

"\nclass NERDataset(Dataset):\n  def __init__(self, encodes, bios):\n    self.encodes = defaultdict(list)\n    for dict_encode in encodes:\n      for k,v in dict_encode.items():\n        self.encodes[k].append(v)\n    self.bios = bios\n\n  def __getitem__(self, idx):\n    item = {k: torch.tensor(v[idx]) for k,v in self.encodes.items()}\n    item['labels'] = torch.tensor(self.bios[idx])\n    return item\n\n  def __len__(self):\n    return len(self.bios)"

データ分割

In [ ]:
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2

In [ ]:
from sklearn.model_selection import train_test_split


list_text_id_train, list_text_id_test, list_bio_id_train, list_bio_id_test = \
    train_test_split(list_text_id, list_bio_id, test_size=test_ratio, random_state=42)
list_text_id_train, list_text_id_valid, list_bio_id_train, list_bio_id_valid = \
    train_test_split(list_text_id_train, list_bio_id_train, test_size=val_ratio/(test_ratio+train_ratio), random_state=42)



データを.txt形式で保存しておく

In [ ]:
!mkdir -p /content/drive/MyDrive/bert/6_2_chABSA/dataset/

import pickle
# データの保存
f = open('/content/drive/MyDrive/bert/6_2_chABSA/dataset/list_text_id_train.txt', 'wb')
pickle.dump(list_text_id_train, f)
f = open('/content/drive/MyDrive/bert/6_2_chABSA/dataset/list_text_id_valid.txt', 'wb')
pickle.dump(list_text_id_valid, f)
f = open('/content/drive/MyDrive/bert/6_2_chABSA/dataset/list_text_id_test.txt', 'wb')
pickle.dump(list_text_id_test, f)

f = open('/content/drive/MyDrive/bert/6_2_chABSA/dataset/list_bio_id_train.txt', 'wb')
pickle.dump(list_bio_id_train, f)
f = open('/content/drive/MyDrive/bert/6_2_chABSA/dataset/list_bio_id_valid.txt', 'wb')
pickle.dump(list_bio_id_valid, f)
f = open('/content/drive/MyDrive/bert/6_2_chABSA/dataset/list_bio_id_test.txt', 'wb')
pickle.dump(list_bio_id_test, f)


# データの読み込み
# f = open('/content/drive/MyDrive/bert/6_2_chABSA/dataset/list_text_id_train.txt', 'rb')
# list_text_id_train = pickle.load(f)
# f = open('/content/drive/MyDrive/bert/6_2_chABSA/dataset/list_text_id_valid.txt', 'rb')
# list_text_id_valid = pickle.load(f)
# f = open('/content/drive/MyDrive/bert/6_2_chABSA/dataset/list_text_id_test.txt', 'rb')
# list_text_id_test = pickle.load(f)

# f = open('/content/drive/MyDrive/bert/6_2_chABSA/dataset/list_bio_id_train.txt', 'rb')
# list_bio_id_train = pickle.load(f)
# f = open('/content/drive/MyDrive/bert/6_2_chABSA/dataset/list_bio_id_valid.txt', 'rb')
# list_bio_id_valid = pickle.load(f)
# f = open('/content/drive/MyDrive/bert/6_2_chABSA/dataset/list_bio_id_test.txt', 'rb')
# list_bio_id_test = pickle.load(f)

In [ ]:
ds_train = NERDataset(list_text_id_train, list_bio_id_train)
ds_valid = NERDataset(list_text_id_valid, list_bio_id_valid)
ds_test = NERDataset(list_text_id_test, list_bio_id_test, is_test=True)

print("学習データ数：", len(ds_train))
print("検証データ数：", len(ds_valid))
print("テストデータ数：", len(ds_test))

学習データ数： 806
検証データ数： 269
テストデータ数： 269


## 固有表現抽出モデルの作成

## 問題1

In [ ]:
# Q. BertForTokenClassificationを使って、事前学習済みモデルを読み込みましょう

from transformers import BertForTokenClassification
model = BertForTokenClassification.from_pretrained(
    pretrained_model_name_or_path=,
    id2label=,
    label2id=)

model.to(device)

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the m

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
!mkdir -p /content/drive/MyDrive/bert/6_2_chABSA/results
from transformers import Trainer, TrainingArguments
import torch

## 問題2

In [ ]:
batch_size = 8

# Q. training_configとtrainerを定義しましょう

training_config = TrainingArguments(
  output_dir='/content/drive/MyDrive/bert/6_2_chABSA/results',
  num_train_epochs=20,
  per_device_train_batch_size=,
  per_device_eval_batch_size=,
  warmup_steps=500,
  weight_decay=0.2,
  save_steps=500,
  do_eval=True,
  eval_steps=100
)

trainer = Trainer(
    model=,
    args=,
    tokenizer=,
    train_dataset=,
    eval_dataset=
)

## 問題3

In [ ]:
# Q. trainerを使って、学習しましょう

 # この行に学習を行うコードを実装しましょう

***** Running training *****
  Num examples = 806
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2020


Step,Training Loss


## テストデータの推論

In [ ]:
# 固有表現抽出の評価用ライブラリをインストール
!pip install seqeval

## 問題4

In [ ]:
# Q. trainerを使って、推論結果をresult変数としましょう

result =  # この行にテストデータで推論を行うコードを実装しましょう

In [ ]:
import numpy as np

trues =  np.vectorize(lambda x:id2label[x])(ds_test.bios_id).tolist()

preds_id = np.argmax(result.predictions, axis=2)
preds = np.vectorize(lambda x:id2label[x])(preds_id).tolist()

In [ ]:
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score

print(f1_score(trues, preds))

print(classification_report(trues, preds))